In [ ]:
import json
import os


from dotenv import load_dotenv
from openai import OpenAI

import pandas as pd

In [224]:
load_dotenv()

True

In [225]:
TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [226]:
openai_client = OpenAI(api_key=OPENAI_API_KEY)

In [227]:
nouns_df = pd.read_csv('not_full_metaphors.csv')

In [228]:

nouns = nouns_df.time.to_list()
metaphors = nouns_df.metaphor.to_list()

nouns[:2], metaphors[:2]



(['year', 'people'],
 ['A year is a book filled with chapters of growth and wisdom.',
  'People are threads woven into the tapestry of life.'])

In [230]:
# prompt_str = """
# You are a linguist expert.
# You ave a file with metaphors. You need to generate a list of metaphors from metaphor column and return add 10-12 metaphors
# according to nouns column to that list. If list is full and there are no more nouns, don't add.

# OUTPUT FORMAT:
# {{"metaphors": ["Жизнь — ржавый замок, который уже не откроется",
# "Надежда — увядший цветок в пустыне", ...]
#  "is_full": false
# }}
# nouns: {}
# metaphors: {}
# """

In [ ]:
# prompt_str = """You are an expert linguist specialized in metaphorical language. 
#     You can answer only with python programming language structures. No quotes no formatting, only pure python.

#     You have an existing list of metaphors stored in the "metaphors" column, and a list of nouns stored in the "nouns" column. Your task is to carefully 
#     generate new, creative, and evocative metaphors using nouns from the "nouns" column and append them to the existing "metaphors" list.

#     Guidelines:

#     Generate between 10 to 20 new unique metaphors, clearly structured as comparisons or vivid images 
#     (e.g., "Life is a rusty lock that will never open again").

#     Only use nouns without metaphors provided in the "nouns" list to create new metaphors. Sort them in desc order according to the nouns list.
#     Align positions of the indexes in the list of metaphors with the indexes of the nouns

#     Provide your answer strictly in the following list format. Output example: '["A year is a book filled with chapters of growth and wisdom.", "A way is a bridge connecting dreams to reality.", ...]'


#     INPUT:
#     nouns: {}
#     metaphors: {}
        
# """

In [ ]:
negative_prompt_str = """You are an expert linguist specialized in metaphorical language. 
    You can answer only with python programming language structures. No quotes no formatting, only pure python.

    You will receive a list on nouns. You must generate a negative metaphor for every noun and 
    return a list with metaphors in exactly the same order. Indexes of nouns and metaphors must be aligned.


    Provide your answer strictly in the following list format. Use only double quotes. Return only a python list, no other symbols,
      no docstring, triple quotes. Only python list.

    EXAMPLE:
        INPUT: ["love", "sun"...]
        OUTPUT: ["Love is a broken compass leading astray", "The sun is a ruthless tyrant in the desert" ...]

    INPUT: {}
         
"""

In [255]:
prompt_str = """You are an expert linguist specialized in metaphorical language. 
    You can answer only with python programming language structures. No quotes no formatting, only pure python.

    You will receive a list on nouns. You must generate a positive metaphor for every noun and 
    return a list with metaphors in exactly the same order. Indexes of nouns and metaphors must be aligned.


    Provide your answer strictly in the following list format. Use only double quotes. Return only a python list, no other symbols,
      no docstring, triple quotes. Only python list.

    EXAMPLE:
        INPUT: ["life", "work"...]
        OUTPUT: ["Life is a journey illuminated by moments of joy and learning", "Work is the melody played by hands shaping dreams" ...]

    INPUT: {}
         
"""

In [264]:
prompt = prompt_str.format(nouns)
prompt

'You are an expert linguist specialized in metaphorical language. \n    You can answer only with python programming language structures. No quotes no formatting, only pure python.\n\n    You will receive a list on nouns. You must generate a positive metaphor for every noun and \n    return a list with metaphors in exactly the same order. Indexes of nouns and metaphors must be aligned.\n\n\n    Provide your answer strictly in the following list format. Use only double quotes. Return only a python list, no other symbols,\n      no docstring, triple quotes. Only python list.\n\n    EXAMPLE:\n        INPUT: ["life", "work"...]\n        OUTPUT: ["Life is a journey illuminated by moments of joy and learning", "Work is the melody played by hands shaping dreams" ...]\n\n    INPUT: [\'year\', \'people\', \'way\', \'man\', \'day\', \'thing\', \'child\', \'Mr\', \'government\', \'work\', \'life\', \'woman\', \'system\', \'case\', \'part\', \'group\', \'number\', \'world\', \'house\', \'area\', \'

In [265]:
completion = openai_client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.5,
)



In [266]:
res_str = completion.choices[0].message.content.strip()
res_str

'["A year is a tapestry woven with threads of hope and renewal", "People are the stars that illuminate the universe of connection", "The way is a river flowing with opportunities and discoveries", "A man is a mountain standing strong against the winds of time", "A day is a canvas painted with colors of possibility and light", "A thing is a seed blossoming into a garden of potential", "A child is a spark igniting the flame of future dreams", "Mr is a beacon of wisdom guiding the path to success", "Government is the orchestra conducting the symphony of society", "Work is the melody played by hands shaping dreams", "Life is a journey illuminated by moments of joy and learning", "A woman is a flower blooming with grace and resilience", "The system is a clockwork ticking in harmony with progress", "A case is a puzzle waiting to reveal its hidden truths", "A part is a note contributing to the harmony of the whole", "A group is a constellation shining brighter together", "A number is a key un

In [ ]:
n = 100
batch_size = 100
n_len = len(nouns)
res_list = []
start_len = 0
my_metaphors = []

while len(nouns) >= n:
    
    my_prompt = prompt_str.format(nouns[(n - batch_size) : n])
    
    completion = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": my_prompt}],
        temperature=0.5,
    )
    res_str = completion.choices[0].message.content
    print(n, res_str)
    res_list = json.loads(res_str)
    
    n += batch_size

    my_metaphors.extend(res_list)
    

100 ["A year is a book filled with chapters of growth and renewal", "People are stars in the vast sky of humanity, shining uniquely", "The way is a river flowing with opportunities and adventures", "A man is a mountain, standing strong and reaching for the sky", "A day is a canvas painted with the colors of possibility", "A thing is a seed, holding the potential for endless creation", "A child is a garden blooming with innocence and wonder", "Mr is a lighthouse guiding ships through the storms of life", "The government is a bridge connecting the past and future", "Work is a symphony composed of dedication and passion", "Life is a tapestry woven with threads of experience and love", "A woman is a river, flowing with strength and grace", "The system is a clockwork, ticking with precision and order", "A case is a puzzle, waiting to be solved with insight", "A part is a note in the melody of a harmonious whole", "A group is a constellation, united by shared purpose", "A number is a key unl

In [ ]:
len(my_metaphors)

In [273]:
my_metaphors[:4]

['A year is a book filled with chapters of growth and renewal',
 'People are stars in the vast sky of humanity, shining uniquely',
 'The way is a river flowing with opportunities and adventures',
 'A man is a mountain, standing strong and reaching for the sky']

In [274]:
nouns_df = nouns_df[:len(my_metaphors)]

In [275]:
# nouns_df["negative_metaphor"] = my_metaphors
# nouns_df.to_csv("negative_metaphors.csv")

nouns_df["positive_metaphor"] = my_metaphors
nouns_df.to_csv("all_metaphors.csv")